XGBoost

Regression

Начало такое же как и в градиентом бустинге

Изначальный лист делаем и строим дерево на residuals

Но строится это дерево по другому

Сначала также делаем initial prediction (0.5 по дефолту и для регрессий и для классификаций)

находим residuals по всем экземплярам

Каждое дерево появляется как лист в который засовываем все residuals

Просчитываем Quality/Similarity score для них

SS = sum of residuals squared / number of residuals + lambda (параметр для регуляризации)

Негативные и позитивные residuals обнулят друг друга (в отличии от squared sum of residuals)

Подсчитали SS score

Теперь мы должны узнать сможем ли мы do a better job clustering similar residuals if we split them into two groups

Берём фичу (кол-во доз например)

Берём первые два observations  (например 10 и 20)

Берём среднее у них (15)

Строим ноду dosage < 15 

И засовываем в листья residuals по всем экземплярам (то есть в левый лист попадут residuals тех экземпляров что < 15, в правый >=

Теперь считаем SS score для этих двух листьев

Получили ноду с двумя листьями  в которых (и ноде и ее листьях) находятся residuals и подсчитали ss score для этих трёх элементов

(Когда residuals в ноде/листе сильно отличаются друг от друга, то они cancel each other out и ss score относительно мал, когда в ноде/листе residuals похожи или он там один, то ss score относительно большой)

Теперь нужно квантифицировать насколько лучше листья cluster similar residuals than the root

Делаем это вычислением Gain (gain of splitting residuals into 2 groups)

Gain = SS левого листа +  SS правого - SS рута

Теперь высчитываем SS и Gain для остальных значений которые мы используем в кач-ве thteshold в ноде (среднее каждых двух соседних значений dosage, те как когда выбираем какое значение continuous фичи выбрать в качестве трешхолда для ноды)

Выбираем трешхолд который нам выдал максимальный Gain

Когда все значения прошли (а если их супер много то как отсеивать?) и выбрали дерево с самым высоким Gain, то если в листе один residual всего, то не трогаем этот лист, если больше , то продолжаем делить

Опять выбираем трешхолд для фичи (значение dosage с которым сравнивать будем) и строим дерево (точнее просто добавляем ещё один уровень в наше дерево)

И также подсчитываем SS и Gain

И так строим обычно до глубины в 6 деревьев, либо по параметру Cover - минимальное кол-во residuals в каждом листе

Cover = знаменатель SS score минус лямбда (то есть number of residuals + lambda - lambda)

Теперь пруним дерево на основании значений Gain

Берём число (гамма тут называется ), например 130

Берём Gain самой низкой ветки дерева и вычитаем из него гамму, если получилось негативное число , то удаляем ветку (в теории все дерево может так удалиться)

Теперь как output value (Y которое дерево выдает) листа высчитывается

OV = sum of residuals/number of residuals + lambda

Те как SS score, но числитель в квадрат не возводится 

Все, построили первое дерево

Делать предсказания также как и в обычном градиентном бустинге

Начальное предсказание (начальный лист) + предсказание дерева умноженное на learning rate

И также строим новые деревья (обучая их на новых residuals) пока residuals не станут супер мелкими или не достигнем максимального кол-ва деревьев


Classification

Сначала также делаем изначальное предсказание (initial prediction)

По дефолту 0.5 ( вероятность что экземпляр относится к классу А, а не Б)

У SS score новая формула

Сумма ряда residuals возведённая в квадрат делить на
Сумма ряда предыдущая вероятность умноженная на (1-предыдущая вероятность) + лямбда (регуляризационный параметр)

Строим дерево

Также берём лист в который все residuals засовываем (1-0.5, если реальный Y класс А, 0-0.5, если Б)

Считаем SS score для этого листа

Теперь должны узнать сможем ли мы лучше кластеризовать (cluster) похожие residuals ,если разделим из на две группы

Также выбираем лучший (с наибольшим Gain) трешхолд по Х

Короч все также как и в регрессии

Cover тут то есть = Сумма ряда предыдущая вероятность умноженная на (1-предыдущая вероятность)

Pruning такой же

Предикты также делаются (+ также как и в обычном градиентном бустинге для классификации мы должны конвертить вероятность в log(odds))